In [3]:
import pulp
import random

Source: https://or.stackexchange.com/questions/11342/need-help-writing-a-metaheuristic-problem/11352#11352

# Data

In [21]:
N= 5
size_1 = 2
size_2 = 3
size_3 = 10

In [22]:
items = [x for x in range(N)]
score = {}
for u in items:
    for v in items:
        if u<v:
            score[(u,v)]=random.choice([_ for _ in range(20)])

In [23]:
subsets = [tuple(c) for c in pulp.combination(items, size_1)]+[tuple(c) for c in pulp.combination(items, size_2)]#+[tuple(c) for c in pulp.combination(items, size_3)]

In [24]:
subsets

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 2, 3),
 (0, 2, 4),
 (0, 3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4)]

In [25]:
subset_score = {}
for s in subsets:
    subset_score[s]=sum(score[(u,v)] for u in s for v in s if  (u,v) in score)

# Optimization

In [26]:
x = pulp.LpVariable.dicts("subset", subsets, cat=pulp.LpBinary)
prob = pulp.LpProblem("grouping_problem", pulp.LpMaximize)
prob += pulp.lpSum([subset_score[s] * x[s] for s in subsets])
for v in items:
    prob += pulp.lpSum(x[s] for s in subsets if v in s)==1, "subset_for_%s"%v

In [27]:
prob

grouping_problem:
MAXIMIZE
5*subset_(0,_1) + 30*subset_(0,_1,_2) + 31*subset_(0,_1,_3) + 26*subset_(0,_1,_4) + 8*subset_(0,_2) + 22*subset_(0,_2,_3) + 12*subset_(0,_2,_4) + 9*subset_(0,_3) + 24*subset_(0,_3,_4) + 3*subset_(0,_4) + 17*subset_(1,_2) + 39*subset_(1,_2,_3) + 36*subset_(1,_2,_4) + 17*subset_(1,_3) + 47*subset_(1,_3,_4) + 18*subset_(1,_4) + 5*subset_(2,_3) + 18*subset_(2,_3,_4) + 1*subset_(2,_4) + 12*subset_(3,_4) + 0
SUBJECT TO
subset_for_0: subset_(0,_1) + subset_(0,_1,_2) + subset_(0,_1,_3)
 + subset_(0,_1,_4) + subset_(0,_2) + subset_(0,_2,_3) + subset_(0,_2,_4)
 + subset_(0,_3) + subset_(0,_3,_4) + subset_(0,_4) = 1

subset_for_1: subset_(0,_1) + subset_(0,_1,_2) + subset_(0,_1,_3)
 + subset_(0,_1,_4) + subset_(1,_2) + subset_(1,_2,_3) + subset_(1,_2,_4)
 + subset_(1,_3) + subset_(1,_3,_4) + subset_(1,_4) = 1

subset_for_2: subset_(0,_1,_2) + subset_(0,_2) + subset_(0,_2,_3)
 + subset_(0,_2,_4) + subset_(1,_2) + subset_(1,_2,_3) + subset_(1,_2,_4)
 + subset_(2,_3) + sub

In [9]:
#prob.writeLP("myproblem.lp")

In [28]:
prob.solve(pulp.PULP_CBC_CMD(timeLimit=10))

1

In [29]:
for s in x:
    if x[s].value() > 0.9:
        print(s)

(0, 2)
(1, 3, 4)
